In [1]:
from cobra.io import read_sbml_model
from mewpy.simulation import get_simulator
import pandas as pd
import matplotlib.pyplot as plt
import cobra
import numpy as np
import csv
from pandas import DataFrame
import statistics
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objs as go
from mewpy.simulation import set_default_solver
set_default_solver('gurobi')

In [2]:
%store -r t_2020_4
expression = t_2020_4.transpose()
%store -r media_only_essential_reactions
%store -r media_only_essential_genes
%store -r DMEM #set per cell line
media = DMEM
growth_rate = 0.01564 #set per cell line
indexes = list(range(0, 64))
for n in range(len((t_2020_4.index).to_list())):
    if 'CAOV3' == ((t_2020_4.index).to_list())[n]: #set per cell line
        t = n
row_number_transcriptomics = t
print('row:',t)


no stored variable or alias #set
no stored variable or alias per
no stored variable or alias cell
no stored variable or alias line
row: 14


In [3]:
t_2020_4.index 

Index(['COV434', '59M', 'NZOV9', 'OAW42', 'COV644', 'TO14', 'COV413A', 'OAW28',
       'JHOC5', 'JHOS4', 'FUOV1', 'OVMANA', 'TYKNU', 'PEO1', 'CAOV3', 'RMGI',
       'BIN67', 'OELE', 'OVK18', 'JHOS2', 'ES2', 'JHOM1', 'EFO27', 'HSKTC',
       'OV7', 'OVSAHO', 'NIHOVCAR3', 'OCIC4P', 'OVISE', 'KURAMOCHI', 'COV362',
       'OVCAR8', 'PA1', 'OC316', 'OCIC5X', 'OV17R', 'SKOV3', 'EFO21',
       'TOV112D', 'OC314', 'SNU119', 'IGROV1', 'A2780', 'SNU840', 'CAOV4',
       'JHOM2B', 'OVCAR4', 'SCCOHT1', 'PEA1', 'OCIP5X', 'OV56', 'ONCODG1',
       'SNU8', 'OVKATE', 'OVTOKO', 'OV90', 'MCAS', 'COV318', 'OVCAR5', 'RMUGS',
       'HEYA8', 'PEO4', 'COV504', 'TOV21G'],
      dtype='object', name='cell_line')

In [4]:
model = read_sbml_model('Human-GEM-annotated.xml')
media_only_simulation = get_simulator(model, envcond = media)
media_only_result = media_only_simulation.simulate()
media_only_result.find(['biomass_human'])

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24


,Flux rate
Reaction ID,
biomass_human,0.213772


In [5]:
expression = expression.transpose()
t = expression
gene_exp = t.iloc[row_number_transcriptomics,:]

g_list = []
for k in gene_exp.keys():
    g_list.append(k)
    
g_list = []
for k in gene_exp.keys():
    g_list.append(k)

g_ensembl = []
for n in range(len(g_list)):
    if 'ENS' not in (g_list[n].split())[0]:
        g_ensembl.append((g_list[n].split())[1])
    if 'ENS' in (g_list[n].split())[0]:
        g_ensembl.append((g_list[n].split())[0])
        
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
g_ensembl_2 = []
for n in range(len(g_ensembl)):
    for x in g_ensembl[n]:
        if x in punc:
            g_ensembl[n] = g_ensembl[n].replace(x, "")
    g_ensembl_2.append(g_ensembl[n])
g_ensembl_2

vs = []
for v in gene_exp:
    vs.append(v)
    
gene_exp_dict = {}
for n in range(len(g_ensembl_2)):
    gene_exp_dict[g_ensembl_2[n]] = vs[n]
    
ANDs = []
ORs = []
ANDORs = []
one_gene = []
no_gene = []

for r in model.reactions:
    if 'and' in r.gene_reaction_rule and 'or' not in r.gene_reaction_rule:
        ANDs.append(r.id)
    if 'and' in r.gene_reaction_rule and 'or' in r.gene_reaction_rule:
        ANDORs.append(r.id)
    if 'or' in r.gene_reaction_rule and 'and' not in r.gene_reaction_rule:
        ORs.append(r.id)
    if len(r.gene_reaction_rule) == 0:
        no_gene.append(r.id)
    if len(r.gene_reaction_rule) != 0:
        if 'or' in r.gene_reaction_rule:
            continue
        elif 'and' in r.gene_reaction_rule:
            continue
        else:
            one_gene.append(r.id)

print('AND rules: ', len(ANDs))
print('ANDOR rules: ', len(ANDORs))
print('OR rules: ', len(ORs))
print('ONE GENE rules: ', len(one_gene))
print('NO GENE rules: ', len(no_gene))
print('Proportion of model not annotated: ', len(no_gene)/len(model.reactions)*100, '%')
print('Proportion of model which IS annotated: ', 100-(len(no_gene)/len(model.reactions)*100), '%')
print('Total Reactions = ', len(model.reactions))
print(len(ORs)+len(ANDs)+len(ANDORs)+len(one_gene)+len(no_gene))

one_gene_forward = []
one_gene_reversible = []
for r in one_gene:
    if model.reactions.get_by_id(r).reversibility == True:
        one_gene_reversible.append(r)
    else:
        one_gene_forward.append(r)
print('number of reversible, one gene reactions =', len(one_gene_reversible))
print('number of forward, one gene reactions =', len(one_gene_forward))

AND rules:  653
ANDOR rules:  129
OR rules:  3972
ONE GENE rules:  3282
NO GENE rules:  5060
Proportion of model not annotated:  38.63775198533904 %
Proportion of model which IS annotated:  61.36224801466096 %
Total Reactions =  13096
13096
number of reversible, one gene reactions = 1074
number of forward, one gene reactions = 2208


In [6]:
media_only_simulation = get_simulator(model, envcond = media)
media_only_result = media_only_simulation.simulate()
print('growth with media constraints only: ', media_only_result.find(['biomass_human']).iloc[0,0],
      'g/gDW/hour','=',(1/media_only_result.find(['biomass_human']).iloc[0,0]),'hours')

growth with media constraints only:  0.21377181121424524 g/gDW/hour = 4.6778852380952385 hours


In [7]:
media_only_result.find(['biomass_human']).iloc[0,0]

0.21377181121424524

In [8]:
1/0.21377181121424524

4.6778852380952385

In [9]:
one_gene_constrained_reactions = {} #create dictionary to contain reaction bounds, according to code below
                                    #this dictionary will be used by MEWpy to constrain model 
for r in one_gene_forward:
    if r not in media.keys(): #do not overwrite media conditions
        if r not in media_only_essential_reactions: #do not constrain essential reactions/genes, or growth will be blocked
            if model.reactions.get_by_id(r).gene_reaction_rule not in media_only_essential_genes:
                if model.reactions.get_by_id(r).gene_reaction_rule in gene_exp_dict.keys(): #check we have expression data
                    if gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0):
                        model.reactions.get_by_id(r).bounds = (0,(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule])) #set upper bound equal to expression value
                        solution = model.optimize() #optimise after constraint, using COBRApy
                        if solution.fluxes['biomass_human'] <= growth_rate: #enforce experimental growth value as minimum gorwth rates constraints can reach
                            print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            model.reactions.get_by_id(r).bounds = (0,1000) #if growth is constrained below experimental, then reopen reaction
                            solution = model.optimize()
                            print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds #save reaction bounds to dictionary
                        if solution.fluxes['biomass_human'] != 0:
                            if solution.fluxes['biomass_human'] > growth_rate:
                                print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds #use expression data as reaction bound, if it doesn't fall below experimental
                            
for r in one_gene_reversible: #run constraints code for reversible, one-gene reactions
    if r not in media.keys():
        if r not in media_only_essential_reactions: #change to new dict
            if model.reactions.get_by_id(r).gene_reaction_rule not in media_only_essential_reactions:
                if model.reactions.get_by_id(r).gene_reaction_rule in gene_exp_dict.keys(): 
                    if gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0): 
                        #here, set lower bound as -1*expression value, and upper bound as positive expression value
                        model.reactions.get_by_id(r).bounds = (-1*(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule]),(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule]))
                        solution = model.optimize()
                        if solution.fluxes['biomass_human'] <= growth_rate:
                            print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            model.reactions.get_by_id(r).bounds = (-1000,1000)
                            solution = model.optimize()
                            print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            #update constraints dictionary with reversible reaction bounds
                            one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                        if solution.fluxes['biomass_human'] != 0:
                            if solution.fluxes['biomass_human'] > growth_rate:
                                print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                            
or_forward = []
or_reversible = []
for r in ORs:
    if model.reactions.get_by_id(r).reversibility == True:
        or_reversible.append(r)
    else:
        or_forward.append(r)
print('number of reversible, OR reactions =', len(or_reversible))
print('number of forward, OR reactions =', len(or_forward))

for r in or_forward:
    if r not in media.keys():
        if r not in media_only_essential_reactions:
            rule_list = [] #make list to append with terms in reaction rule
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = [] #append new list with only ensembl IDs from reaction rule
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #calculate sum of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            sum_of_expressions = sum(exp_list)
            #set bounds according to sum of expressions
            model.reactions.get_by_id(r).bounds = (0,sum_of_expressions) #same as 'one-gene' method, where expression (sum) is upper bound
            solution = model.optimize()
            if solution.fluxes['biomass_human'] <= growth_rate:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (0,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] != 0:
                if solution.fluxes['biomass_human'] > growth_rate:
                    print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                
for r in or_reversible:
    if r not in media.keys():
        if r not in media_only_essential_reactions:
            #find ensembl ids in gene-reaction rules
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #calculate sum of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            sum_of_expressions = sum(exp_list)
            #set bounds according to sum of expressions; same as before, use negative and positive expression (sum) as bounds
            model.reactions.get_by_id(r).bounds = (-1*sum_of_expressions,sum_of_expressions)
            solution = model.optimize()
            if solution.fluxes['biomass_human'] <= growth_rate:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (-1000,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] != 0:
                if solution.fluxes['biomass_human'] > growth_rate:
                    print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                
and_forward = []
and_reversible = []
for r in ANDs:
    if model.reactions.get_by_id(r).reversibility == True:
        and_reversible.append(r)
    else:
        and_forward.append(r)
print('number of reversible, AND reactions =', len(and_reversible))
print('number of forward, AND reactions =', len(and_forward))

for r in and_forward:
    if r not in media.keys():
        if r not in media_only_essential_reactions:
            #find ensembl IDs in reaction rules, as performed for 'or' rules
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #generate list of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            if len(exp_list) > 1:
                exp_list.sort()
                min_expression = exp_list[0]
            if len(exp_list) == 1:
                min_expression = exp_list[0] #set variable for minimum, rate-limiting subunit expression
            #use minimum expression as upper bound for forward reactions
            model.reactions.get_by_id(r).bounds = (0,min_expression)
            solution = model.optimize()
            if solution.fluxes['biomass_human'] <= growth_rate:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (0,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] != 0:
                if solution.fluxes['biomass_human'] > growth_rate:
                    print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
for r in and_reversible:
    if r not in media.keys():
        if r not in media_only_essential_reactions:
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #generate list of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            if len(exp_list) > 1:
                exp_list.sort()
                min_expression = exp_list[0]
            if len(exp_list) == 1:
                min_expression = exp_list[0]
            #set bounds according to minimum expression; use negative and positive expression (minimum) as bounds
            model.reactions.get_by_id(r).bounds = ((-1*min_expression),min_expression)
            solution = model.optimize()
            if solution.fluxes['biomass_human'] <= growth_rate:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (-1000,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] != 0:
                if solution.fluxes['biomass_human'] > growth_rate:
                    print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                
all_and_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions)
all_and_result = all_and_simulation.simulate()
print('growth with media AND gene expression constraints: ', all_and_result.find(['biomass_human']).iloc[0,0],
      'g/gDW/hour','=',(1/all_and_result.find(['biomass_human']).iloc[0,0]),'hours')

HMR_3907 : (0, 6.277426896532659) 0.21377181121424524
HMR_4097 : (0, 4.219555769166913) 0.21377181121424524
HMR_4108 : (0, 4.219555769166913) 0.21377181121424524
HMR_4133 : (0, 4.219555769166913) 0.21377181121424524
HMR_4360 : (0, 2.729008870337863) 0.21377181121424524
HMR_4372 : (0, 4.450881315273369) 0.21377181121424524
HMR_7747 : (0, 5.634593268445757) 0.21377181121424524
HMR_8360 : (0, 4.173926931999809) 0.21377181121424524
HMR_8757 : (0, 3.96624587322494) 0.21377181121424524
HMR_5397 : (0, 5.613531652917927) 0.21377181121424524
HMR_5399 : (0, 3.404630684217612) 0.21377181121424524
HMR_5400 : (0, 3.404630684217612) 0.21377181121424524
HMR_8592 : (0, 5.156639310692087) 0.21377181121424524
HMR_8589 : (0, 5.156639310692087) 0.21377181121424524
HMR_8583 : (0, 0.01435529297707) 0.21377181121424524
HMR_8584 : (0, 0.0) 0.21377181121424524
HMR_8585 : (0, 2.611172380044005) 0.21377181121424524
HMR_3944 : (0, 6.481395824154128) 0.21377181121424524
HMR_4132 : (0, 3.736604875314249) 0.21377181

In [10]:
caov3_260923 = all_and_result.dataframe #set per cell line
%store caov3_260923 #set per cell line

Stored 'caov3_260923' (DataFrame)


UsageError: Unknown variable '#set'


In [11]:
caov3_260923_model = model
%store caov3_260923_model

Stored 'caov3_260923_model' (Model)


In [12]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

caov3_deletion_results = single_gene_deletion(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24
Read LP format model from file /var/folders/5c/xx9bql4j4s9c930jy0hsxb1rnlqn5f/T/tmp6a1yv_nv.lp
Reading time = 0.11 seconds
: 8400 rows, 26192 columns, 111634 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24
Read LP format model from file /var/folders/5c/xx9bql4j4s9c930jy0hsxb1rnlqn5f/T/tmpu47u6lck.lp
Reading time = 0.28 seconds
: 8400 rows, 26192 columns, 111634 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24
Read LP format model from file /var/folders/5c/xx9bql4j4s9c930jy0hsxb1rnlqn5f/T/tmpfuzyq3mt.lp
Reading time = 0.35 seconds
: 8400 rows, 26192 columns, 111634 nonzeros


In [14]:
%store caov3_deletion_results

Stored 'caov3_deletion_results' (DataFrame)
